## 0. Libarary 불러오기

In [71]:
import os, glob
import time
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

from torchsummary import summary

from sklearn.model_selection import train_test_split

In [66]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda:0


## 1. 데이터셋 정의

### (1) Train Dataset

In [2]:
# 학습 데이터셋 폴더 경로를 지정해주세요.
train_dir = '/opt/ml/input/data/train'
trainimage_dir = os.path.join(train_dir, 'images')

In [3]:
# meta 데이터와 이미지 경로를 불러옵니다.
train_df = pd.read_csv(os.path.join(train_dir, 'train.csv'))
train_df

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59
...,...,...,...,...,...
2695,006954,male,Asian,19,006954_male_Asian_19
2696,006955,male,Asian,19,006955_male_Asian_19
2697,006956,male,Asian,19,006956_male_Asian_19
2698,006957,male,Asian,20,006957_male_Asian_20


In [5]:
masks = ['mask1', 'mask2', 'mask3', 'mask4', 'mask5', 'incorrect_mask', 'normal']
mask_df = pd.DataFrame()
for person in train_df.values:
    for mask in masks:
        mask_df = mask_df.append(pd.Series(np.append(person, mask)), ignore_index=True)
mask_df.columns = np.append(train_df.columns.values, 'mask')
mask_df

,id,gender,race,age,path,mask
0,000001,female,Asian,45.0,000001_female_Asian_45,mask1
1,000001,female,Asian,45.0,000001_female_Asian_45,mask2
2,000001,female,Asian,45.0,000001_female_Asian_45,mask3
3,000001,female,Asian,45.0,000001_female_Asian_45,mask4
4,000001,female,Asian,45.0,000001_female_Asian_45,mask5
...,...,...,...,...,...,...
18895,006959,male,Asian,19.0,006959_male_Asian_19,mask3
18896,006959,male,Asian,19.0,006959_male_Asian_19,mask4
18897,006959,male,Asian,19.0,006959_male_Asian_19,mask5
18898,006959,male,Asian,19.0,006959_male_Asian_19,incorrect_mask


In [49]:
mask_df = mask_df.sample(frac=1).reset_index(drop=True)
mask_df

,id,gender,race,age,path,mask
0,001224,female,Asian,24.0,001224_female_Asian_24,mask4
1,006093,male,Asian,19.0,006093_male_Asian_19,mask5
2,005047,female,Asian,26.0,005047_female_Asian_26,mask1
3,001174,male,Asian,25.0,001174_male_Asian_25,mask5
4,003409,female,Asian,56.0,003409_female_Asian_56,mask3
...,...,...,...,...,...,...
18895,001657,female,Asian,18.0,001657_female_Asian_18,normal
18896,003755,male,Asian,52.0,003755_male_Asian_52,normal
18897,003553,female,Asian,47.0,003553_female_Asian_47,mask2
18898,001639,male,Asian,18.0,001639_male_Asian_18,mask1


In [53]:
train, valid = train_test_split(mask_df, test_size=0.2)
print(f'Train Set dim : (%d, %d)' % (train.shape))
print(f'Valid Set dim : (%d, %d)' % (valid.shape))

Train Set dim : (15120, 6)
Valid Set dim : (3780, 6)


In [54]:
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])

In [79]:
class GenderDataset(Dataset):
    def __init__(self, path, mask_df, transform):
        super(GenderDataset).__init__()
        self.path = path
        self.mask_df = mask_df
        self.transform = transform
        
    def __getitem__(self, idx):
        full_path = os.path.join(self.path, self.mask_df.iloc[idx]['path'])
        img_list = glob.glob(full_path + '/*')
        file_name = self.mask_df.iloc[idx]['mask']
        for img_name in img_list:
            if img_name.startswith(file_name):
                break
        image = Image.open(os.path.join(full_path, img_name))
        if self.transform:
            image = self.transform(image)
        
        label = self.mask_df.iloc[idx]['gender']
        label = 0 if label=='male' else 1
        return image, label
    
    def __len__(self):
        return len(self.mask_df)

In [80]:
gender_train_data = GenderDataset(trainimage_dir, train, transform)
gender_valid_data = GenderDataset(trainimage_dir, valid, transform)

In [81]:
gender_train = DataLoader(gender_train_data, batch_size=32, shuffle=True, num_workers=2)
gender_valid = DataLoader(gender_valid_data, batch_size=32, shuffle=True, num_workers=2)

In [82]:
class AgeDataset(Dataset):
    def __init__(self, path, mask_df, transform):
        self.path = path
        self.mask_df = mask_df
        self.transform = transform
        
    def __getitem__(self, idx):
        full_path = os.path.join(self.path, self.mask_df.iloc[idx]['path'])
        img_list = glob.glob(full_path + '/*')
        file_name = self.mask_df.iloc[idx]['mask']
        for img_name in img_list:
            if img_name.startswith(file_name):
                break
        image = Image.open(os.path.join(full_path, img_name))
        if self.transform:
            image = self.transform(image)
        
        label = self.mask_df.iloc[idx]['age']
        if label >= 60.0:
            label = 2
        elif label >= 30.0:
            label = 1
        else:
            label = 0
        return image, label
    
    def __len__(self):
        return len(self.mask_df)

In [83]:
age_train_data = AgeDataset(trainimage_dir, train, transform)
age_valid_data = AgeDataset(trainimage_dir, valid, transform)

In [84]:
age_train = DataLoader(age_train_data, batch_size=32, shuffle=True, num_workers=2)
age_valid = DataLoader(age_valid_data, batch_size=32, shuffle=True, num_workers=2)

In [85]:
class MaskDataset(Dataset):
    def __init__(self, path, mask_df, transform):
        self.path = path
        self.mask_df = mask_df
        self.transform = transform
        
    def __getitem__(self, idx):
        full_path = os.path.join(self.path, self.mask_df.iloc[idx]['path'])
        img_list = glob.glob(full_path + '/*')
        file_name = self.mask_df.iloc[idx]['mask']
        for img_name in img_list:
            if img_name.startswith(file_name):
                break
        image = Image.open(os.path.join(full_path, img_name))
        if self.transform:
            image = self.transform(image)
        
        label = self.mask_df.iloc[idx]['mask']
        if label.startswith('mask'):
            label = 0
        elif label.startswith('incorrect'):
            label = 1
        else:
            label = 2
        return image, label
    
    def __len__(self):
        return len(self.mask_df)

In [86]:
mask_train_data = MaskDataset(trainimage_dir, train, transform)
mask_valid_data = MaskDataset(trainimage_dir, valid, transform)

In [87]:
mask_train = DataLoader(mask_train_data, batch_size=32, shuffle=True)
mask_valid = DataLoader(mask_valid_data, batch_size=32, shuffle=True)

### (2) Test Dataset

In [111]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [112]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

In [113]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
testimage_dir = os.path.join(test_dir, 'images')

In [114]:
# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(testimage_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
test_dataset = TestDataset(image_paths, transform)

test_loader = DataLoader(
    test_dataset,
    shuffle=False
)

## 2. Model 정의

In [94]:
class MyModel(nn.Module):
    def __init__(self, num_classes: int = 1000):
        super(MyModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(64, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [95]:
model = MyModel(num_classes=3)

In [96]:
model.to('cuda')

MyModel(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=64, out_features=32, bias=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=32, out_features=3, bias=True)
  )
)

In [97]:
summary(model, (3, 512, 384))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 64, 127, 95]          23,296
       BatchNorm2d-2          [-1, 64, 127, 95]             128
              ReLU-3          [-1, 64, 127, 95]               0
 AdaptiveAvgPool2d-4             [-1, 64, 1, 1]               0
           Dropout-5                   [-1, 64]               0
            Linear-6                   [-1, 32]           2,080
              ReLU-7                   [-1, 32]               0
            Linear-8                    [-1, 3]              99
Total params: 25,603
Trainable params: 25,603
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 2.25
Forward/backward pass size (MB): 17.67
Params size (MB): 0.10
Estimated Total Size (MB): 20.02
----------------------------------------------------------------


### 마스크 착용 여부 분류

In [103]:
mask_model = MyModel(num_classes=3)
mask_model.to(device)

MyModel(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=64, out_features=32, bias=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=32, out_features=3, bias=True)
  )
)

### 성별 분류

In [104]:
gender_model = MyModel(num_classes=2)
gender_model.to(device)

MyModel(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=64, out_features=32, bias=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=32, out_features=2, bias=True)
  )
)

### 나이 분류

In [105]:
age_model = MyModel(num_classes=3)
age_model.to(device)

MyModel(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=64, out_features=32, bias=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=32, out_features=3, bias=True)
  )
)

## 3. Training

### 마스크 착용 여부 분류

In [106]:
optimizer = optim.Adam(mask_model.parameters(), lr=1e-3)
lr_sched = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

num_epochs = 30

valid_early_stop = 0
valid_best_loss = float('inf')
EARLY_STOPPING_EPOCH = 5
since = time.time()

final_train_loss = []
final_train_acc = []
final_valid_loss = []
final_valid_acc = []

mask_model.train()
for e in range(num_epochs) :
    print(f' ====================== epoch %d ======================' % (e+1) )
    train_loss_list = []
    train_acc_list = []

    # train
    for i, (images, targets) in enumerate(mask_train) : 
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        scores = mask_model(images)
        _, preds = scores.max(dim=1)

        loss = F.cross_entropy(scores, targets)
        loss.backward()
        optimizer.step()

        correct = sum(targets == preds).cpu()
        acc=(correct/32 * 100)

        train_loss_list.append(loss)
        train_acc_list.append(acc)

        if i % 50 == 0 :
            print(f'Iteration %3.d | Train Loss  %.4f | Classifier Accuracy %2.2f' % (i, loss, acc))

    train_mean_loss = np.mean(train_loss_list, dtype="float64")
    train_mean_acc = np.mean(train_acc_list, dtype="float64")

    final_train_loss.append(train_mean_loss)
    final_train_acc.append(train_mean_acc)

    epoch_time = time.time() - since
    since = time.time()

    print('')
    print(f'[Summary] Elapsed time : %.0f m %.0f s' % (epoch_time // 60, epoch_time % 60))
    print(f'Train Loss Mean %.4f | Accuracy %2.2f ' % (train_mean_loss, train_mean_acc) )

    # validation 
    valid_loss_list = []
    valid_acc_list = []
    for i, (images, targets) in enumerate(mask_valid) : 
        optimizer.zero_grad()
        images = images.to(device=device)
        targets = targets.to(device=device)

        with torch.no_grad():
            scores = mask_model(images)
            loss = F.cross_entropy(scores, targets)
            _, preds = scores.max(dim=1)

        correct = sum(targets == preds).cpu()
        acc=(correct/32 * 100)

        valid_loss_list.append(loss)
        valid_acc_list.append(acc)

    val_mean_loss = np.mean(valid_loss_list, dtype="float64")
    val_mean_acc = np.mean(valid_acc_list, dtype="float64")

    final_valid_loss.append(val_mean_loss)
    final_valid_acc.append(val_mean_acc)

    print(f'Valid Loss Mean %.4f | Accuracy %2.2f ' % (val_mean_loss, val_mean_acc) )
    print('')

    if val_mean_loss < valid_best_loss:
        valid_best_loss = val_mean_loss
        valid_early_stop = 0
        # new best model save (valid 기준)
        best_model = mask_model
        path = './mask_model/'
        torch.save(best_model.state_dict(), f'{path}model{val_mean_acc:2.2f}_epoch_{e}.pth')
    else:
        # early stopping    
        valid_early_stop += 1
        if valid_early_stop >= EARLY_STOPPING_EPOCH:
            print("EARLY STOPPING!!")
            break

    lr_sched.step()

 ====================== epoch 1 ======================
Iteration   0 | Train Loss  1.0962 | Classifier Accuracy 34.38
Iteration  50 | Train Loss  0.8587 | Classifier Accuracy 68.75
Iteration 100 | Train Loss  0.9022 | Classifier Accuracy 68.75
Iteration 150 | Train Loss  0.9320 | Classifier Accuracy 65.62
Iteration 200 | Train Loss  0.7116 | Classifier Accuracy 78.12
Iteration 250 | Train Loss  0.8411 | Classifier Accuracy 71.88
Iteration 300 | Train Loss  0.6701 | Classifier Accuracy 81.25
Iteration 350 | Train Loss  0.7773 | Classifier Accuracy 71.88
Iteration 400 | Train Loss  0.9211 | Classifier Accuracy 65.62
Iteration 450 | Train Loss  0.8691 | Classifier Accuracy 65.62

[Summary] Elapsed time : 1 m 44 s
Train Loss Mean 0.8161 | Accuracy 71.07 
Valid Loss Mean 0.8053 | Accuracy 70.77 

 ====================== epoch 2 ======================
Iteration   0 | Train Loss  0.9341 | Classifier Accuracy 62.50
Iteration  50 | Train Loss  0.7782 | Classifier Accuracy 71.88
Iteration 100 | 

### 성별 분류

In [108]:
optimizer = optim.Adam(gender_model.parameters(), lr=1e-3)
lr_sched = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

num_epochs = 30

valid_early_stop = 0
valid_best_loss = float('inf')
EARLY_STOPPING_EPOCH = 5
since = time.time()

final_train_loss = []
final_train_acc = []
final_valid_loss = []
final_valid_acc = []

gender_model.train()
for e in range(num_epochs) :
    print(f' ====================== epoch %d ======================' % (e+1) )
    train_loss_list = []
    train_acc_list = []

    # train
    for i, (images, targets) in enumerate(gender_train) : 
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        scores = gender_model(images)
        _, preds = scores.max(dim=1)

        loss = F.cross_entropy(scores, targets)
        loss.backward()
        optimizer.step()

        correct = sum(targets == preds).cpu()
        acc=(correct/32 * 100)

        train_loss_list.append(loss)
        train_acc_list.append(acc)

        if i % 50 == 0 :
            print(f'Iteration %3.d | Train Loss  %.4f | Classifier Accuracy %2.2f' % (i, loss, acc))

    train_mean_loss = np.mean(train_loss_list, dtype="float64")
    train_mean_acc = np.mean(train_acc_list, dtype="float64")

    final_train_loss.append(train_mean_loss)
    final_train_acc.append(train_mean_acc)

    epoch_time = time.time() - since
    since = time.time()

    print('')
    print(f'[Summary] Elapsed time : %.0f m %.0f s' % (epoch_time // 60, epoch_time % 60))
    print(f'Train Loss Mean %.4f | Accuracy %2.2f ' % (train_mean_loss, train_mean_acc) )

    # validation 
    valid_loss_list = []
    valid_acc_list = []
    for i, (images, targets) in enumerate(gender_valid) : 
        optimizer.zero_grad()
        images = images.to(device=device)
        targets = targets.to(device=device)

        with torch.no_grad():
            scores = gender_model(images)
            loss = F.cross_entropy(scores, targets)
            _, preds = scores.max(dim=1)

        correct = sum(targets == preds).cpu()
        acc=(correct/32 * 100)

        valid_loss_list.append(loss)
        valid_acc_list.append(acc)

    val_mean_loss = np.mean(valid_loss_list, dtype="float64")
    val_mean_acc = np.mean(valid_acc_list, dtype="float64")

    final_valid_loss.append(val_mean_loss)
    final_valid_acc.append(val_mean_acc)

    print(f'Valid Loss Mean %.4f | Accuracy %2.2f ' % (val_mean_loss, val_mean_acc) )
    print('')

    if val_mean_loss < valid_best_loss:
        valid_best_loss = val_mean_loss
        valid_early_stop = 0
        # new best model save (valid 기준)
        gender_best_model = gender_model
        path = './gender_model/'
        torch.save(best_model.state_dict(), f'{path}model{val_mean_acc:2.2f}_epoch_{e}.pth')
    else:
        # early stopping    
        valid_early_stop += 1
        if valid_early_stop >= EARLY_STOPPING_EPOCH:
            print("EARLY STOPPING!!")
            break

    lr_sched.step()

 ====================== epoch 1 ======================
Iteration   0 | Train Loss  0.6924 | Classifier Accuracy 53.12
Iteration  50 | Train Loss  0.6987 | Classifier Accuracy 59.38
Iteration 100 | Train Loss  0.5753 | Classifier Accuracy 78.12
Iteration 150 | Train Loss  0.6142 | Classifier Accuracy 68.75
Iteration 200 | Train Loss  0.6173 | Classifier Accuracy 68.75
Iteration 250 | Train Loss  0.6409 | Classifier Accuracy 65.62
Iteration 300 | Train Loss  0.6745 | Classifier Accuracy 53.12
Iteration 350 | Train Loss  0.6861 | Classifier Accuracy 59.38
Iteration 400 | Train Loss  0.5974 | Classifier Accuracy 68.75
Iteration 450 | Train Loss  0.6200 | Classifier Accuracy 65.62

[Summary] Elapsed time : 1 m 8 s
Train Loss Mean 0.6585 | Accuracy 61.36 
Valid Loss Mean 0.6621 | Accuracy 60.14 

 ====================== epoch 2 ======================
Iteration   0 | Train Loss  0.6700 | Classifier Accuracy 53.12
Iteration  50 | Train Loss  0.6859 | Classifier Accuracy 62.50
Iteration 100 | T

### 나이 분류

In [109]:
optimizer = optim.Adam(age_model.parameters(), lr=1e-3)
lr_sched = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

num_epochs = 30

valid_early_stop = 0
valid_best_loss = float('inf')
EARLY_STOPPING_EPOCH = 5
since = time.time()

final_train_loss = []
final_train_acc = []
final_valid_loss = []
final_valid_acc = []

age_model.train()
for e in range(num_epochs) :
    print(f' ====================== epoch %d ======================' % (e+1) )
    train_loss_list = []
    train_acc_list = []

    # train
    for i, (images, targets) in enumerate(age_train) : 
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        scores = age_model(images)
        _, preds = scores.max(dim=1)

        loss = F.cross_entropy(scores, targets)
        loss.backward()
        optimizer.step()

        correct = sum(targets == preds).cpu()
        acc=(correct/32 * 100)

        train_loss_list.append(loss)
        train_acc_list.append(acc)

        if i % 50 == 0 :
            print(f'Iteration %3.d | Train Loss  %.4f | Classifier Accuracy %2.2f' % (i, loss, acc))

    train_mean_loss = np.mean(train_loss_list, dtype="float64")
    train_mean_acc = np.mean(train_acc_list, dtype="float64")

    final_train_loss.append(train_mean_loss)
    final_train_acc.append(train_mean_acc)

    epoch_time = time.time() - since
    since = time.time()

    print('')
    print(f'[Summary] Elapsed time : %.0f m %.0f s' % (epoch_time // 60, epoch_time % 60))
    print(f'Train Loss Mean %.4f | Accuracy %2.2f ' % (train_mean_loss, train_mean_acc) )

    # validation 
    valid_loss_list = []
    valid_acc_list = []
    for i, (images, targets) in enumerate(age_valid) : 
        optimizer.zero_grad()
        images = images.to(device=device)
        targets = targets.to(device=device)

        with torch.no_grad():
            scores = age_model(images)
            loss = F.cross_entropy(scores, targets)
            _, preds = scores.max(dim=1)

        correct = sum(targets == preds).cpu()
        acc=(correct/32 * 100)

        valid_loss_list.append(loss)
        valid_acc_list.append(acc)

    val_mean_loss = np.mean(valid_loss_list, dtype="float64")
    val_mean_acc = np.mean(valid_acc_list, dtype="float64")

    final_valid_loss.append(val_mean_loss)
    final_valid_acc.append(val_mean_acc)

    print(f'Valid Loss Mean %.4f | Accuracy %2.2f ' % (val_mean_loss, val_mean_acc) )
    print('')

    if val_mean_loss < valid_best_loss:
        valid_best_loss = val_mean_loss
        valid_early_stop = 0
        # new best model save (valid 기준)
        age_best_model = age_model
        path = './age_model/'
        torch.save(best_model.state_dict(), f'{path}model{val_mean_acc:2.2f}_epoch_{e}.pth')
    else:
        # early stopping    
        valid_early_stop += 1
        if valid_early_stop >= EARLY_STOPPING_EPOCH:
            print("EARLY STOPPING!!")
            break

    lr_sched.step()

 ====================== epoch 1 ======================
Iteration   0 | Train Loss  1.1152 | Classifier Accuracy 28.12
Iteration  50 | Train Loss  0.7981 | Classifier Accuracy 65.62
Iteration 100 | Train Loss  0.9373 | Classifier Accuracy 53.12
Iteration 150 | Train Loss  0.7393 | Classifier Accuracy 65.62
Iteration 200 | Train Loss  0.9226 | Classifier Accuracy 50.00
Iteration 250 | Train Loss  0.9060 | Classifier Accuracy 50.00
Iteration 300 | Train Loss  0.8675 | Classifier Accuracy 56.25
Iteration 350 | Train Loss  0.7638 | Classifier Accuracy 59.38
Iteration 400 | Train Loss  0.9923 | Classifier Accuracy 56.25
Iteration 450 | Train Loss  0.6774 | Classifier Accuracy 65.62

[Summary] Elapsed time : 1 m 7 s
Train Loss Mean 0.8460 | Accuracy 59.35 
Valid Loss Mean 0.8052 | Accuracy 63.21 

 ====================== epoch 2 ======================
Iteration   0 | Train Loss  0.6856 | Classifier Accuracy 68.75
Iteration  50 | Train Loss  0.8595 | Classifier Accuracy 56.25
Iteration 100 | T

## 4. Inference

### 마스크 착용 여부 분류

In [123]:
best_model.eval()
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
mask_predictions = []
for images in test_loader:
    with torch.no_grad():
        images = images.to(device)
        scores = best_model(images)
        preds = scores.argmax(dim=-1)
        mask_predictions.extend(preds.cpu().numpy())

In [124]:
len(mask_predictions)

12600

In [125]:
from collections import Counter
Counter(mask_predictions)

Counter({0: 12600})

* 마스크는 그냥 Wear 로 예측하도록 학습된 것 같다.

### 성별 분류

In [120]:
gender_best_model.eval()
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
gender_predictions = []
for images in test_loader:
    with torch.no_grad():
        images = images.to(device)
        scores = gender_best_model(images)
        preds = scores.argmax(dim=-1)
        gender_predictions.extend(preds.cpu().numpy())

In [121]:
len(gender_predictions)

12600

In [122]:
from collections import Counter
Counter(gender_predictions)

Counter({1: 11160, 0: 1440})

* 성별은 학습 데이터셋의 여성/남성 비율 차이에 비해 너무 여성으로만 예측한 것 같다.

### 나이 분류

In [126]:
age_best_model.eval()
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
age_predictions = []
for images in test_loader:
    with torch.no_grad():
        images = images.to(device)
        scores = age_best_model(images)
        preds = scores.argmax(dim=-1)
        age_predictions.extend(preds.cpu().numpy())

In [127]:
len(age_predictions)

12600

In [128]:
from collections import Counter
Counter(age_predictions)

Counter({1: 8775, 0: 3825})

* 60대 이상은 예측되지 않았다.
* 이상하게 30대 이하가 많이 나올 줄 알았는데 30대 이상 60대 미만이 많이 나왔다.
* 범위 조절을 잘못했을 가능성도 있을 것 같다.

### 최종 클래스 선정

In [130]:
all_predictions = []
size = len(submission)
class_map = np.array([[[0, 1, 2],
                       [3, 4, 5]],
                      [[6, 7, 8],
                       [9, 10, 11]],
                      [[12, 13, 14],
                       [15, 16, 17]]])
for idx in range(size):
    i = mask_predictions[idx]
    j = gender_predictions[idx]
    k = age_predictions[idx]
    all_predictions.append(class_map[i][j][k])

In [132]:
len(all_predictions)

12600

In [133]:
from collections import Counter
Counter(all_predictions)

Counter({4: 8382, 3: 2778, 0: 1047, 1: 393})

* 충격적이다... 4개 클래스만 나오다니...

In [134]:
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission_baseline.csv'), index=False)
print('test inference is done!')

test inference is done!
